In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
4665,2024-02-20,Eua Ncaa,21:00,UMass,VCU Rams,1.78,2.01,142.5,1.89,1.89,-3.5,2.09,1.64,Ee4JHTcn,0.561798,0.497512,0.529101,0.529101,0.059310,130.122,40.929746,0.314549,1.8,1.643168,0.912871,110.97,1.780,0.610819,0.343157,6.0,122.166,23.313424,0.190834,2.4,1.341641,0.559017,137.75,1.828,0.412456,0.225632,33.0,81,95,1.37,1.45,106.968,126.718,0.085823,0.000000,0.170616,0.20,0.040,19.500000,0.694423,0.8,0.105577,1.39,0.278,3.633094,0.000000,0.0,0.000000
4666,2024-02-20,Eua Ncaa,21:00,West Virginia,UCF Knights,2.22,1.65,141.5,1.83,1.91,1.5,2.04,1.69,S6Wmd3ur,0.450450,0.606061,0.546448,0.523560,0.056511,298.512,127.450064,0.426951,0.6,1.341641,2.236068,242.19,4.464,2.241167,0.502054,-74.0,230.390,149.700648,0.649771,0.6,1.341641,2.236068,141.34,3.080,1.576753,0.511933,-8.0,81,74,2.99,1.91,264.466,335.146,0.208295,0.030251,0.132701,5.11,1.022,1.193738,0.421707,0.5,0.078293,-0.56,-0.112,-5.803571,0.000000,0.0,0.000000
4667,2024-02-20,Eua Ncaa,21:00,Western Michigan,Miami (Ohio),2.01,1.78,140.5,1.89,1.89,-1.5,2.10,1.65,6DJGGtIg,0.497512,0.561798,0.529101,0.529101,0.059310,219.384,103.275342,0.470751,0.0,0.000000,NaN,125.58,3.794,1.498709,0.395021,-96.0,187.110,84.680023,0.452568,1.2,1.643168,1.369306,138.40,2.820,1.188423,0.421427,-7.0,42,80,2.99,1.73,133.508,205.238,0.085823,0.000000,0.169706,0.38,0.076,13.289474,0.504314,0.4,-0.104314,1.02,0.204,3.823529,0.284492,0.3,0.015508
4668,2024-02-20,Eua Ncaa,22:00,Northern Illinois,Ball State,1.86,1.92,145.5,1.89,1.89,-2.5,2.06,1.66,v3BTDv2C,0.537634,0.520833,0.529101,0.529101,0.058468,161.760,69.110213,0.427239,1.8,1.643168,0.912871,263.20,2.742,1.749949,0.638202,-30.0,140.220,15.196776,0.108378,0.6,1.341641,2.236068,123.90,2.028,0.094446,0.046571,-33.0,47,59,5.60,2.10,140.032,140.482,0.022448,0.000000,0.152066,-2.37,-0.474,-1.814346,0.000000,0.0,0.000000,0.64,0.128,7.187500,0.373347,0.4,0.026653
4669,2024-02-20,Eua Ncaa,22:30,Creighton,UConn,2.20,1.66,145.5,1.82,1.90,1.5,2.02,1.70,E1o7SVvS,0.454545,0.602410,0.549451,0.526316,0.056955,131.272,23.826059,0.181501,1.8,1.643168,0.912871,133.51,1.534,0.396522,0.258489,46.0,100.694,17.773786,0.176513,3.0,0.000000,0.000000,110.97,1.214,0.249960,0.205898,111.0,79,81,1.69,1.37,99.546,109.256,0.197843,0.030413,0.121653,-0.35,-0.070,-17.142857,0.802987,0.7,-0.102987,1.79,0.358,1.843575,0.747594,0.8,0.052406
4670,2024-02-20,Eua Ncaa,23:00,Brigham Young,Baylor,1.66,2.20,155.5,1.85,1.89,-4.5,2.03,1.68,0KUefsAf,0.602410,0.454545,0.540541,0.529101,0.056955,107.148,13.734749,0.128185,1.8,1.643168,0.912871,111.22,1.368,0.271238,0.198273,-5.0,136.782,50.402496,0.368488,2.4,1.341641,0.559017,128.78,1.884,1.019034,0.540888,35.0,83,94,1.34,1.37,117.524,147.078,0.197843,0.015125,0.133416,0.26,0.052,12.692308,0.816723,0.8,-0.016723,-1.99,-0.398,-3.015075,0.000000,0.0,0.000000
4671,2024-02-20,Eua Ncaa,23:00,Texas Tech,TCU,1.45,2.71,147.5,1.84,1.90,-7.5,2.06,1.66,6cVieNfl,0.689655,0.369004,0.543478,0.526316,0.058659,164.108,76.766416,0.467780,1.2,1.643168,1.369306,273.80,2.238,1.013543,0.452879,19.0,132.226,40.066102,0.303012,1.8,1.643168,0.912871,150.00,1.880,0.817863,0.435034,3.0,74,75,3.70,2.00,113.296,180.976,0.428344,0.022688,0.152066,1.17,0.234,1.923077,0.773150,0.9,0.126850,1.16,0.232,7.370690,0.000000,0.0,0.000000
4672,2024-02-20,Polônia Liga Feminina

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
4690,08:30,China Wcba Feminina,Shanxi Xing Rui F,Jiangsu Phoenix F,1.82,Back Home
